## Clasificacion de las personas en situacion de pobreza

https://public.tableau.com/app/profile/andy.kriebel/viz/VisualVocabulary/VisualVocabulary

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split #, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix, jaccard_score, log_loss

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# random forest

import warnings
warnings.simplefilter("ignore")

In [4]:
dta = pd.read_csv("./data/data_f.csv", index_col=0)

dta.head()

,edad,hombre,est_sup,colegio_priv,gran_empresa,segundo_trab,ingbrut,urbano,no_pobre
1,28.0,0.0,1.0,0.0,0.0,1.0,1454.500000,1.0,1.0
2,48.0,0.0,0.0,0.0,0.0,1.0,2037.000000,1.0,1.0
3,25.0,0.0,0.0,0.0,0.0,0.0,2037.000000,1.0,1.0
4,16.0,0.0,0.0,0.0,0.0,0.0,2037.000000,1.0,1.0
6,41.0,0.0,0.0,0.0,0.0,1.0,1842.066732,1.0,1.0


### Visualizaciones

Primero, es necesario realizar algunos graficos para las variables que son continuas como en el caso de `ingbrut` y `edad`

In [ ]:
ax = sns.displot(dta, x="ingbrut", binwidth=10)

ax.set(title="Distribucion de los ingresos")
ax.set_axis_labels("Ingresos brutos", "Frecuencia", fontsize = 12)

plt.savefig("./figures/distribucion_ingbrut.pdf")

In [ ]:
ax = sns.displot(dta, x="edad", binwidth=10, palette="Set1")

ax.set(title="Distribucion de las edades")
ax.set_axis_labels("Edad", "Frecuencia", fontsize = 12)

plt.savefig("./figures/distribucion_edad.pdf", bbox_inches="tight", transparent=True)

Como se observa en el primer grafico, es necesario realizar una correccion a las cifras de ingresos brutos, debido a que esta concentrado principalmente en valores muy bajos. Esto se realizara aplicando logaritmos a las variables.

In [5]:
dta = dta.loc[(dta["ingbrut"] > 0)]

dta["ln_ing"] = np.log(dta["ingbrut"])

In [ ]:
ax = sns.displot(dta, x="ln_ing")

ax.set(title="Distribucion de los log Ingresos")
ax.set_axis_labels("Log Ingresos", "Frecuencia", fontsize = 12)

plt.savefig("./figures/distribucion_ln_ing.pdf", bbox_inches="tight", transparent=True)

Una vez corregida la variable de ingresos, e incluso habiendola vuelto mas parecida a una normal, se puede realizar un grafico de dispersion entre los `ingresos` y la variable `no pobre`

In [ ]:
ax = sns.lmplot(x="ln_ing", y="no_pobre", data=dta, logistic=True, line_kws={"color": "red"})

ax.set(title="Dispersion de la pobreza por ingresos")
ax.set_axis_labels("Log Ingresos", "No pobre", fontsize = 12)

plt.savefig("./figures/distribucion_ln_ing.pdf", bbox_inches="tight", transparent=True)

### Procesamiento: split y estandarizacion
Primero definimos las variables que tendremos como explicativas y al target

In [19]:
X_names = ["edad", "hombre", "est_sup", "colegio_priv", "gran_empresa", "segundo_trab", "urbano", "ln_ing"]
X = dta[X_names]

y = dta["no_pobre"]

Estandarizamos cada valor de la explicativa

In [20]:
X = StandardScaler().fit_transform(X)
X

array([[-0.81102386, -1.07773763,  2.03392515, ...,  1.96795541,
         0.68294492, -0.18201526],
       [ 0.37979517, -1.07773763, -0.49166018, ...,  1.96795541,
         0.68294492,  0.14456907],
       [-0.98964671, -1.07773763, -0.49166018, ..., -0.5081416 ,
         0.68294492,  0.14456907],
       ...,
       [-0.3942372 ,  0.92786961,  2.03392515, ..., -0.5081416 ,
         0.68294492, -0.29619951],
       [-0.03699149, -1.07773763, -0.49166018, ..., -0.5081416 ,
         0.68294492, -0.0253364 ],
       [-1.16826956,  0.92786961, -0.49166018, ..., -0.5081416 ,
         0.68294492, -0.0253364 ]])

Ahora se separará la base de datos entre las de `entrenamiento` y `testeo`

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(f"La cantidad de observaciones que tiene el training set es {y_train.shape[0]}\
 y la de test {y_test.shape[0]}")

La cantidad de observaciones que tiene el training set es 60301 y la de test 20101


### Prueba 1: Modelo lineal

In [22]:
lm = LinearRegression().fit(X_train, y_train)
lm

LinearRegression()

In [23]:
print(f"Tras la regresión, el coeficiente es {np.round(lm.intercept_, 2)}\
 y la lista de coeficientes es el siguiente:\n\
 {X_names}\n {np.round(lm.coef_, 3)}")

Tras la regresión, el coeficiente es 0.83 y la lista de coeficientes es el siguiente:
 ['edad', 'hombre', 'est_sup', 'colegio_priv', 'gran_empresa', 'segundo_trab', 'urbano', 'ln_ing']
 [ 0.037 -0.004  0.006  0.019  0.001  0.003  0.046  0.115]


Aunque tambien se podria observar los resultados mediante la libreria de `statsmodels`

In [24]:
ols = sm.OLS(y_train, X_train).fit()
print(ols.summary())

                                 OLS Regression Results                                
Dep. Variable:               no_pobre   R-squared (uncentered):                   0.029
Model:                            OLS   Adj. R-squared (uncentered):              0.029
Method:                 Least Squares   F-statistic:                              226.9
Date:                Wed, 26 Oct 2022   Prob (F-statistic):                        0.00
Time:                        18:10:16   Log-Likelihood:                         -79225.
No. Observations:               60301   AIC:                                  1.585e+05
Df Residuals:                   60293   BIC:                                  1.585e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Con un modelo lineal, el `ajuste es muy bajo`

In [25]:
r2 = lm.score(X_train, y_train)
print(f"El score de ajuste es {np.round(r2, 4)}")

El score de ajuste es 0.1744


In [26]:
y_pred = lm.predict(X_test)
print(f"El score de ajuste es {accuracy_score(y_test, y_pred)}")

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
### ajuste

### Prueba 2: Modelo Logit 

In [ ]:
### solo usaré las variables que sean significativas

In [32]:
X_names = ["edad", "hombre", "est_sup", "colegio_priv", "urbano", "ln_ing"]
X = dta[X_names]
y = dta["no_pobre"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [33]:
logit = sm.Logit(y_train, X_train).fit()
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.385731
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               no_pobre   No. Observations:                60301
Model:                          Logit   Df Residuals:                    60295
Method:                           MLE   Df Model:                            5
Date:                Wed, 26 Oct 2022   Pseudo R-squ.:                  0.1394
Time:                        18:11:27   Log-Likelihood:                -23260.
converged:                       True   LL-Null:                       -27027.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
edad             0.0028      0.001      4.521      0.000       0.002       0.004
hombre          -0.2344

Los resultados son interesantes:

* Las personas con mayor edad tienen menos probabilidad de ser probreza
* El ser hombre no reduce el nivel de pobreza, sino lo vuelve mas probable
* Tener un estudio superior (no universitario, universitario, maestria o doctorado) reduce los niveles de pobreza
* Haber estudiado en un colegio privado reduce la probabilidad de ser pobre
* Vivir en una zona urbana tambien reduce la probabilidad de ser pobre
* _El mas evidente_: recibir mayores ingresos tambien reduce la probabilidad de ser pobre

### Modelos de clasificacion

Tras haber definido cuales son las variables que usare, procedo a estimar cada uno de los algoritmos de clasificacion para ver cual se ajusta mejor

#### 1. Logistic Regression

In [ ]:
### ajuste

#### 2. KNN: K-Nearest Neighbours

In [ ]:
### ajuste

#### 3. Decision Tree

In [ ]:
### ajuste

#### 4. Random Forest

In [ ]:
### ajuste

#### Selección del modelo

In [ ]:
### Validación: K